<a href="https://colab.research.google.com/github/MinamiAoyama18/expense-tracker/blob/main/Copy_of_comfyUI_Flux.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install all required dependencies at the start
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install gradio tqdm requests pyngrok

# Get ngrok auth token (you'll need this only once)
from pyngrok import ngrok
ngrok.set_auth_token("2saJsRVhjp6vRBcwWfc1hL7vrBz_7JUtXi9MuT4Ck9HPwdiTLN")  # Replace with your ngrok token

Looking in indexes: https://download.pytorch.org/whl/cu121
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.5/780.5 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 80.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 52.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 116.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 40.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB

In [ ]:
import os
import requests
from tqdm.notebook import tqdm

def download_file(url, dest_path):
    response = requests.get(url, stream=True)
    total_size = int(response.headers.get('content-length', 0))

    with open(dest_path, 'wb') as f, tqdm(
        desc=os.path.basename(dest_path),
        total=total_size,
        unit='iB',
        unit_scale=True,
        unit_divisor=1024,
    ) as pbar:
        for data in response.iter_content(chunk_size=1024):
            size = f.write(data)
            pbar.update(size)

# Define paths
clip_folder = '/content/ComfyUI/models/clip'
fp16_filename = 't5xxl_fp16.safetensors'
fp16_path = os.path.join(clip_folder, fp16_filename)

# URL for the fp16 CLIP model
fp16_url = "https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/text_encoder_2/model.safetensors"

# Create clip folder if it doesn't exist
os.makedirs(clip_folder, exist_ok=True)

# Download fp16 model if it doesn't exist
if not os.path.exists(fp16_path):
    print(f"Downloading {fp16_filename}...")
    download_file(fp16_url, fp16_path)
    print("Download complete!")
else:
    print(f"{fp16_filename} already exists in {clip_folder}")

t5xxl_fp16.safetensors:   0%|          | 0.00/2.59G [00:00<?, ?iB/s]

Download complete!


In [ ]:
import os
import glob

def verify_comfyui_models():
    # Define base path
    base_path = '/content/ComfyUI/models'

    # Define expected models and their locations
    expected_models = {
        'unet': {
            'path': 'unet',
            'files': ['flux1-dev.safetensors']
        },
        'clip': {
            'path': 'clip',
            'files': ['clip_l.safetensors', 't5xxl_fp16.safetensors']
        },
        'vae': {
            'path': 'vae',
            'files': ['ae.safetensors']
        }
    }

    print("=== ComfyUI Model Verification ===\n")

    all_correct = True
    missing_files = []

    # Check each model type
    for model_type, info in expected_models.items():
        full_path = os.path.join(base_path, info['path'])
        print(f"\nChecking {model_type.upper()} models in: {full_path}")

        # Check if directory exists
        if not os.path.exists(full_path):
            print(f"❌ Directory missing: {full_path}")
            os.makedirs(full_path, exist_ok=True)
            print(f"📁 Created directory: {full_path}")
            all_correct = False
            continue

        # Check each expected file
        for file in info['files']:
            file_path = os.path.join(full_path, file)
            if os.path.exists(file_path):
                size = os.path.getsize(file_path) / (1024 * 1024 * 1024)  # Convert to GB
                print(f"✅ Found {file} ({size:.2f} GB)")
            else:
                print(f"❌ Missing: {file}")
                missing_files.append((model_type, file))
                all_correct = False

        # List any unexpected files
        all_files = glob.glob(os.path.join(full_path, "*.safetensors"))
        unexpected = [os.path.basename(f) for f in all_files
                     if os.path.basename(f) not in info['files']]
        if unexpected:
            print("ℹ️ Unexpected files found:", unexpected)

    print("\n=== Summary ===")
    if all_correct:
        print("✅ All required models are in place!")
    else:
        print("❌ Some models are missing:")
        for model_type, file in missing_files:
            print(f"  - {model_type}: {file}")

        print("\nRequired Model Locations:")
        print("- flux1-dev.safetensors → ComfyUI/models/unet/")
        print("- clip_l.safetensors → ComfyUI/models/clip/")
        print("- t5xxl_fp16.safetensors → ComfyUI/models/clip/")
        print("- ae.safetensors → ComfyUI/models/vae/")

# Run the verification
verify_comfyui_models()

=== ComfyUI Model Verification ===


Checking UNET models in: /content/ComfyUI/models/unet
❌ Directory missing: /content/ComfyUI/models/unet
📁 Created directory: /content/ComfyUI/models/unet

Checking CLIP models in: /content/ComfyUI/models/clip
❌ Missing: clip_l.safetensors
✅ Found t5xxl_fp16.safetensors (2.59 GB)

Checking VAE models in: /content/ComfyUI/models/vae
❌ Directory missing: /content/ComfyUI/models/vae
📁 Created directory: /content/ComfyUI/models/vae

=== Summary ===
❌ Some models are missing:
  - clip: clip_l.safetensors

Required Model Locations:
- flux1-dev.safetensors → ComfyUI/models/unet/
- clip_l.safetensors → ComfyUI/models/clip/
- t5xxl_fp16.safetensors → ComfyUI/models/clip/
- ae.safetensors → ComfyUI/models/vae/


In [ ]:
# Install dependencies first
!pip install pyngrok

import os
import shutil
import torch
from tqdm.notebook import tqdm
import time
from subprocess import Popen
from pyngrok import ngrok

# Function definitions remain the same...
# (previous code until the ngrok setup)

# Setup ngrok and start server
print("\nSetting up ComfyUI server with public access...")

# Get ngrok token from user input for security
from getpass import getpass
auth_token = getpass("Enter your ngrok auth token: ")
ngrok.set_auth_token(auth_token)

# Start ComfyUI server
print("Starting ComfyUI server on port 8188...")
process = Popen(['python', 'main.py', '--listen', '--port', '8188'])

# Give the server a moment to start
print("Waiting for server to initialize...")
time.sleep(10)

try:
    # Create public URL using ngrok
    print("Creating public URL...")
    tunnel = ngrok.connect(8188)
    public_url = tunnel.public_url
    print(f"\nComfyUI is now available at: {public_url}")
    print("\nKeeping server running... Use this URL to access ComfyUI")
    print("(Press Ctrl+C or stop the cell execution to stop the server)")

    # Keep the script running
    while True:
        time.sleep(1)

except Exception as e:
    print(f"Error creating tunnel: {e}")
    print("Stopping ComfyUI server...")
    process.terminate()
    raise
finally:
    # Cleanup
    ngrok.disconnect(tunnel.public_url)


Setting up ComfyUI server with public access...
Starting ComfyUI server on port 8188...
Waiting for server to initialize...
Creating public URL...

ComfyUI is now available at: https://6925-35-222-159-153.ngrok-free.app

Keeping server running... Use this URL to access ComfyUI
(Press Ctrl+C or stop the cell execution to stop the server)


In [ ]:
import os
import shutil
from pathlib import Path

def cleanup_comfyui_installations():
    print("=== ComfyUI Cleanup Script ===\n")

    # Define the target directory for the clean installation
    target_dir = '/content/ComfyUI'

    # Find all ComfyUI directories
    comfy_dirs = []
    for root, dirs, _ in os.walk('/content'):
        for dir in dirs:
            if dir == 'ComfyUI':
                path = os.path.join(root, dir)
                comfy_dirs.append(path)

    print(f"Found {len(comfy_dirs)} ComfyUI installations:")
    for dir in comfy_dirs:
        print(f"- {dir}")

    # Identify the most complete installation
    def check_installation_completeness(path):
        required_dirs = ['models', 'web', 'comfy']
        score = sum(1 for dir in required_dirs if os.path.exists(os.path.join(path, dir)))
        has_models = False
        if os.path.exists(os.path.join(path, 'models')):
            model_files = list(Path(os.path.join(path, 'models')).rglob('*.safetensors'))
            has_models = len(model_files) > 0
        return score + (5 if has_models else 0)

    if comfy_dirs:
        best_install = max(comfy_dirs, key=check_installation_completeness)
        print(f"\nMost complete installation found at: {best_install}")

        # Backup models before cleanup
        models_backup = {}
        if os.path.exists(os.path.join(best_install, 'models')):
            print("\nBacking up model files...")
            for model_file in Path(os.path.join(best_install, 'models')).rglob('*.safetensors'):
                models_backup[str(model_file)] = str(model_file.relative_to(best_install))

        # Create fresh target directory
        if os.path.exists(target_dir):
            print(f"\nRemoving existing directory at {target_dir}")
            shutil.rmtree(target_dir)

        # Move the best installation to target directory
        print(f"Moving {best_install} to {target_dir}")
        shutil.move(best_install, target_dir)

        # Clean up other installations
        for dir in comfy_dirs:
            if dir != best_install and os.path.exists(dir):
                print(f"Removing redundant installation: {dir}")
                shutil.rmtree(dir)

        # Clean up empty nested directories
        def remove_empty_dirs(path):
            for root, dirs, files in os.walk(path, topdown=False):
                for dir in dirs:
                    dir_path = os.path.join(root, dir)
                    if not os.listdir(dir_path):
                        os.rmdir(dir_path)
                        print(f"Removed empty directory: {dir_path}")

        remove_empty_dirs('/content')

        print("\n=== Cleanup Complete ===")
        print(f"Clean ComfyUI installation is now at: {target_dir}")

        # Report model locations
        print("\nModel files found:")
        for orig_path, rel_path in models_backup.items():
            print(f"- {rel_path}")
    else:
        print("No ComfyUI installations found!")

# Run the cleanup
cleanup_comfyui_installations()